In [61]:
import pandas as pd
df = pd.read_csv('https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2024/vra_2024_07.csv', sep=';')

## Entendimento de colunas
- Código DI: Indica a etapa do voo realizada, as explicações de cada etapa podem ser encontradas em um ofício do Diário oficial da União, que está salvo em PDF nesse projeto.
- Código Tipo Linha: Indica se um voo é nacional, internacional, de carga ou misto.
- Número Voo:

In [62]:
codigo_di_dict = {'0': 'Padrão',
    '1': 'Etapa extra, tem transporte de carga, mas não era esperado existir essa etapa',
    '2': 'Etapa extra, tem transporte de carga, mas não era esperado existir essa etapa',
    '3': 'Etapa remunerada que não concretizou o transporte',
    '6': 'Não remunerados sem transporte de carga',
    '7': 'Voo fretado',
    '9': 'Etapa de voo Charter',
    'D': 'Voo utilizado para transporte de objetos de um segundo voo',
    'E': 'Voo grátis oferecido pela compainha áerea'}

tipo_de_linha_dict = {
    'N': ''' Doméstica Mista: para operações de transporte aéreo de passageiros
ou mistas, em que todos os aeródromos envolvidos estejam situados simultaneamente
em território brasileiro''',
    'C': '''Doméstica Cargueira: para operações de transporte aéreo exclusivo de
carga e/ou correio em que todos os aeródromos envolvidos estejam situados
simultaneamente em território brasileiro''',
    'I': '''Internacional Mista: para operações de transporte aéreo de passageiros
ou mistas, em que ao menos um dos aeródromos envolvidos esteja situado em território
estrangeiro''',
    'G': '''Internacional Cargueira: para operações de transporte aéreo exclusivo
de carga e/ou correio em que ao menos um dos aeródromos envolvidos esteja situado
em território estrangeiro'''
}

codigos_di_manter = (df['Código DI'].isin(['0', '2']))
tipo_de_linha_manter = df['Código Tipo Linha'].isin(['N', 'I'])
df = df[codigos_di_manter]
df = df[tipo_de_linha_manter]

C:\Users\brjjo287\AppData\Local\Temp\ipykernel_8164\4206395637.py:29: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[tipo_de_linha_manter]


## Análise valores nulos
- Quase todo Voo com partida Real ou Chegada Real nula, foram cancelados

In [63]:
# Análise valores nulos
# Identificar se existe alguma correlação entre esses valores nulos e locais espécificos
display(df.isna().sum())
display(df[(df['Partida Real'].isna()) | ((df['Chegada Real'].isna()))].value_counts('Situação Voo'))

Sigla ICAO Empresa Aérea           0
Número Voo                         0
Código DI                          0
Código Tipo Linha                  0
Sigla ICAO Aeroporto Origem        0
Partida Prevista                 431
Partida Real                    2666
Sigla ICAO Aeroporto Destino       0
Chegada Prevista                 431
Chegada Real                    2667
Situação Voo                       0
dtype: int64

Situação Voo
CANCELADO    2666
REALIZADO       1
Name: count, dtype: int64

## Tratamento dos dados

In [64]:
df = df.dropna(subset=['Partida Prevista', 'Chegada Prevista'])
df = df.drop(columns=['Código DI'])
# 'Código Tipo Linha'
# Removendo voos realizados sem informação de partida e chegada
df = df[~(((df['Partida Real'].isna()) | ((df['Chegada Real'].isna()))) & (df['Situação Voo'] == 'REALIZADO'))]

## Criação Target

In [65]:
df['Partida Prevista'] = pd.to_datetime(df['Partida Prevista'], format="%d/%m/%Y %H:%M")
df['Partida Real'] = pd.to_datetime(df['Partida Real'], format="%d/%m/%Y %H:%M")
df['Chegada Prevista'] = pd.to_datetime(df['Chegada Prevista'], format="%d/%m/%Y %H:%M")
df['Chegada Real'] = pd.to_datetime(df['Chegada Real'], format="%d/%m/%Y %H:%M")
df['Diff partida minutos'] = (df['Partida Real'] - df['Partida Prevista']).dt.total_seconds()/60
df['Diff chegada minutos'] = (df['Chegada Real'] - df['Chegada Prevista']).dt.total_seconds()/60
df['Diff partida minutos'] = df['Diff partida minutos'].round()
df['Diff chegada minutos'] = df['Diff chegada minutos'].round()
df['Diff partida dias'] = (df['Partida Real'] - df['Partida Prevista']).dt.total_seconds()/86400
df['Diff chegada dias'] = (df['Chegada Real'] - df['Chegada Prevista']).dt.total_seconds()/86400
df['Diff partida dias'] = df['Diff partida dias'].round()
df['Diff chegada dias'] = df['Diff chegada dias'].round()

# df = df[~((df['Diff partida dias'] < -1) & (df['Situação Voo'] == 'REALIZADO'))]

In [66]:
df

,Sigla ICAO Empresa Aérea,Número Voo,Código Tipo Linha,Sigla ICAO Aeroporto Origem,Partida Prevista,Partida Real,Sigla ICAO Aeroporto Destino,Chegada Prevista,Chegada Real,Situação Voo,Diff partida minutos,Diff chegada minutos,Diff partida dias,Diff chegada dias
0,AAL,904,I,SBGL,2024-07-01 23:00:00,2024-07-01 22:58:00,KMIA,2024-07-02 07:35:00,2024-07-02 07:34:00,REALIZADO,-2.0,-1.0,-0.0,-0.0
1,AAL,905,I,KMIA,2024-07-01 21:55:00,2024-07-01 21:31:00,SBGL,2024-07-02 06:00:00,2024-07-02 05:55:00,REALIZADO,-24.0,-5.0,-0.0,-0.0
2,AAL,906,I,SBGR,2024-07-01 23:15:00,2024-07-01 23:11:00,KMIA,2024-07-02 07:45:00,2024-07-02 07:21:00,REALIZADO,-4.0,-24.0,-0.0,-0.0
3,AAL,929,I,KMIA,2024-07-01 20:50:00,2024-07-02 12:01:00,SBGR,2024-07-02 05:15:00,2024-07-02 21:15:00,REALIZADO,911.0,960.0,1.0,1.0
4,AAL,930,I,SBGR,2024-07-01 21:25:00,2024-07-01 21:09:00,KMIA,2024-07-02 05:55:00,2024-07-02 05:34:00,REALIZADO,-16.0,-21.0,-0.0,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87747,UAL,149,I,KEWR,2024-07-31 21:30:00,2024-07-31 21:49:00,SBGR,2024-08-01 07:10:00,2024-08-01 08:44:00,REALIZADO,19.0,94.0,0.0,0.0
87748,UAL,844,I,SBGR,2024-07-31 22:20:00,2024-07-31 22:26:00,KORD,2024-08-01 08:50:00,2024-08-01 08:36:00,REALIZADO,6.0,-14.0,0.0,-0.0
87749,UAL,845,I,KORD,2024-07-31 23:15:00,2024-07-31 23:33:00,SBGR,2024-08-01 09:30:00,2024-08-01 10:24:00,REALIZADO,18.0,54.0,0.0,0.0
87750,UAL,860,I,SBGR,2024-07-31 21:30:00,2024-07-31 23:21:00,KIAD,2024-08-01 07:25:00,2024-08-01 08:58:00,REALIZADO,111.0,93.0,0.0,0.0


In [67]:
import seaborn as sns
df[['Diff partida minutos', 'Diff chegada minutos', 'Diff partida dias', 'Diff chegada dias', 
    'Diff partida minutos', 'Diff chegada minutos']].describe(percentiles=[0.05, 0.8, 0.9, 0.95])

,Diff partida minutos,Diff chegada minutos,Diff partida dias,Diff chegada dias,Diff partida minutos,Diff chegada minutos
count,7.813500e+04,78135.000000,78135.000000,78135.000000,7.813500e+04,78135.000000
mean,-2.459657e+01,0.406054,-0.020209,-0.000346,-2.459657e+01,0.406054
std,7.541320e+03,78.120471,5.237077,0.051718,7.541320e+03,78.120471
min,-2.103807e+06,-2880.000000,-1461.000000,-2.000000,-2.103807e+06,-2880.000000
5%,-1.400000e+01,-23.000000,-0.000000,0.000000,-1.400000e+01,-23.000000
50%,-2.000000e+00,-5.000000,0.000000,0.000000,-2.000000e+00,-5.000000
80%,1.000000e+01,9.000000,0.000000,0.000000,1.000000e+01,9.000000
90%,2.300000e+01,22.000000,0.000000,0.000000,2.300000e+01,22.000000
95%,4.300000e+01,41.000000,0.000000,0.000000,4.300000e+01,41.000000
max,2.877000e+03,2856.000000,2.000000,2.000000,2.877000e+03,2856.000000


In [57]:
#df[~df['Partida Real'].isna()].sort_values('Diff partida')
remocao_voos_adiantados
#df[df['Diff partida'] < -1440].sort_values('Diff partida')


,Sigla ICAO Empresa Aérea,Número Voo,Código Tipo Linha,Sigla ICAO Aeroporto Origem,Partida Prevista,Partida Real,Sigla ICAO Aeroporto Destino,Chegada Prevista,Chegada Real,Situação Voo,Diff partida,Diff chegada,Diff partida segundos,Diff chegada segundos,Diff partida dias,Diff chegada dias
55707,JAT,757,I,SBGL,2024-07-31 19:10:00,2020-07-31 19:43:00,SCEL,2024-07-31 23:50:00,2024-07-31 23:01:00,REALIZADO,-2103807.0,-49.0,-1460.977083,-0.034028,-1460.977083,-0.034028
54891,IBE,6824,I,SBGR,2024-07-03 14:20:00,2024-04-03 14:39:00,LEMD,2024-07-04 00:45:00,2024-07-04 00:33:00,REALIZADO,-131021.0,-12.0,-90.986806,-0.008333,-90.986806,-0.008333
87271,TVR,4716,I,VHHH,2024-07-30 04:00:00,2024-07-28 04:00:00,VCBI,2024-07-30 09:00:00,2024-07-28 09:00:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87267,TVR,4716,I,FNLU,2024-07-24 20:00:00,2024-07-22 20:00:00,SBGR,2024-07-25 03:20:00,2024-07-23 03:20:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87266,TVR,4716,I,VCBI,2024-07-23 22:00:00,2024-07-21 22:00:00,FNLU,2024-07-24 06:30:00,2024-07-22 06:30:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87265,TVR,4716,I,VHHH,2024-07-23 04:00:00,2024-07-21 04:00:00,VCBI,2024-07-23 09:00:00,2024-07-21 09:00:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87262,TVR,4716,I,VHHH,2024-07-16 04:00:00,2024-07-14 04:00:00,VCBI,2024-07-16 09:00:00,2024-07-14 09:00:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87261,TVR,4716,I,VCBI,2024-07-16 22:00:00,2024-07-14 22:00:00,FNLU,2024-07-17 06:30:00,2024-07-15 06:30:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87260,TVR,4716,I,FNLU,2024-07-17 20:00:00,2024-07-15 20:00:00,SBGR,2024-07-18 03:20:00,2024-07-16 03:20:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
87258,TVR,4716,I,VCBI,2024-07-03 22:00:00,2024-07-01 22:00:00,FNLU,2024-07-04 06:30:00,2024-07-02 06:30:00,REALIZADO,-2880.0,-2880.0,-2.000000,-2.000000,-2.000000,-2.000000
